# [과제 2] 회귀분석
### - Ch 1, Ch 2를 토대로 자유롭게 회귀분석과 회귀진단을 진행해주세요.
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

In [1]:
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

# Data 

데이터 출처 : https://www.kaggle.com/avikasliwal/used-cars-price-prediction 

< y > 
* **Price** : The price of the used car in INR Lakhs.



< X > 
* Name : The brand and model of the car
* Location : The location in which the car is being sold or is available for purchase.
* Year : The year or edition of the model.
* Kilometers_Driven : The total kilometres driven in the car by the previous owner(s) in KM.
* Fuel_Type : The type of fuel used by the car. (Petrol, Diesel, Electric, CNG, LPG)
* Transmission : The type of transmission used by the car. (Automatic / Manual)
* Owner_Type : Whether the ownership is Firsthand, Second hand or other.
* Mileage : The standard mileage offered by the car company in kmpl or km/kg
* Engine : The displacement volume of the engine in CC.
* Power : The maximum power of the engine in bhp.
* Seats : The number of seats in the car.
* New_Price : The price of a new car of the same model.

In [2]:
# Load Data 
data = pd.read_csv("assignment2_data.csv")
data.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [3]:
data.shape

(6019, 14)

In [4]:
# 필요없는 변수 drop 하기 

final = data.drop(['Unnamed: 0', 'New_Price','Name','Year'], axis=1)

In [5]:
# 변수 이름에 있는 string을 제거한다.
import re

final['Mileage'] = final['Mileage'].map(lambda x: re.sub(r"[^0-9.-]", "", str(x)))
final['Engine'] = final['Engine'].map(lambda x: re.sub(r"[^0-9.-]", "", str(x)))
final['Power'] = final['Power'].map(lambda x: re.sub(r"[^0-9.-]", "", str(x)))

final.head(2)

,Location,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Mumbai,72000,CNG,Manual,First,26.6,998,58.16,5.0,1.75
1,Pune,41000,Diesel,Manual,First,19.67,1582,126.2,5.0,12.50


In [6]:
# na 값 drop 하기 

final2 = final.dropna(axis=0)
final2

,Location,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Mumbai,72000,CNG,Manual,First,26.6,998,58.16,5.0,1.75
1,Pune,41000,Diesel,Manual,First,19.67,1582,126.2,5.0,12.50
2,Chennai,46000,Petrol,Manual,First,18.2,1199,88.7,5.0,4.50
3,Chennai,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,6.00
4,Coimbatore,40670,Diesel,Automatic,Second,15.2,1968,140.8,5.0,17.74
...,...,...,...,...,...,...,...,...,...,...
6014,Delhi,27365,Diesel,Manual,First,28.4,1248,74,5.0,4.75
6015,Jaipur,100000,Diesel,Manual,First,24.4,1120,71,5.0,4.00
6016,Jaipur,55000,Diesel,Manual,Second,14.0,2498,112,8.0,2.90
6017,Kolkata,46000,Petrol,Manual,First,18.9,998,67.1,5.0,2.65


In [7]:
final2.info() # object 함수 변환해야함

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5977 entries, 0 to 6018
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Location           5977 non-null   object 
 1   Kilometers_Driven  5977 non-null   int64  
 2   Fuel_Type          5977 non-null   object 
 3   Transmission       5977 non-null   object 
 4   Owner_Type         5977 non-null   object 
 5   Mileage            5977 non-null   object 
 6   Engine             5977 non-null   object 
 7   Power              5977 non-null   object 
 8   Seats              5977 non-null   float64
 9   Price              5977 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 513.6+ KB


In [42]:
final2['Mileage'] = pd.to_numeric(final2['Mileage'])
final2['Engine'] = pd.to_numeric(final2['Engine'])
final2['Power'] = pd.to_numeric(final2['Power'])


In [9]:
fianl2 = final2.dropna()

In [43]:
final2

,Location,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Mumbai,72000,CNG,Manual,First,26.60,998,58.16,5.0,1.75
1,Pune,41000,Diesel,Manual,First,19.67,1582,126.20,5.0,12.50
2,Chennai,46000,Petrol,Manual,First,18.20,1199,88.70,5.0,4.50
3,Chennai,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,6.00
4,Coimbatore,40670,Diesel,Automatic,Second,15.20,1968,140.80,5.0,17.74
...,...,...,...,...,...,...,...,...,...,...
6014,Delhi,27365,Diesel,Manual,First,28.40,1248,74.00,5.0,4.75
6015,Jaipur,100000,Diesel,Manual,First,24.40,1120,71.00,5.0,4.00
6016,Jaipur,55000,Diesel,Manual,Second,14.00,2498,112.00,8.0,2.90
6017,Kolkata,46000,Petrol,Manual,First,18.90,998,67.10,5.0,2.65


In [52]:
cat_col = ['Location', 'Fuel_Type', 'Transmission', 'Owner_Type']
cat_data = final2[cat_col]

In [53]:
# one_hot encoding 

cat_feature = pd.get_dummies(cat_data,prefix=cat_col)
cat_feature

,Location_Ahmedabad,Location_Bangalore,Location_Chennai,Location_Coimbatore,Location_Delhi,Location_Hyderabad,Location_Jaipur,Location_Kochi,Location_Kolkata,Location_Mumbai,...,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_LPG,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Owner_Type_First,Owner_Type_Fourth & Above,Owner_Type_Second,Owner_Type_Third
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
6015,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,1,0,0,0
6016,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,1,0
6017,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,1,0,0,0


In [55]:
num_cols = ['Kilometers_Driven', 'Mileage', 'Engine', 'Power']
num_feature= final2[num_cols]

In [57]:
# merge
fin_final = pd.concat([cat_feature, num_feature], axis=1)

In [58]:
final_target = final2.Price

In [59]:
fin_final

,Location_Ahmedabad,Location_Bangalore,Location_Chennai,Location_Coimbatore,Location_Delhi,Location_Hyderabad,Location_Jaipur,Location_Kochi,Location_Kolkata,Location_Mumbai,...,Transmission_Automatic,Transmission_Manual,Owner_Type_First,Owner_Type_Fourth & Above,Owner_Type_Second,Owner_Type_Third,Kilometers_Driven,Mileage,Engine,Power
0,0,0,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,72000,26.60,998,58.16
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,41000,19.67,1582,126.20
2,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,46000,18.20,1199,88.70
3,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,87000,20.77,1248,88.76
4,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,40670,15.20,1968,140.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,0,0,0,0,1,0,0,0,0,0,...,0,1,1,0,0,0,27365,28.40,1248,74.00
6015,0,0,0,0,0,0,1,0,0,0,...,0,1,1,0,0,0,100000,24.40,1120,71.00
6016,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,55000,14.00,2498,112.00
6017,0,0,0,0,0,0,0,0,1,0,...,0,1,1,0,0,0,46000,18.90,998,67.10


# 데이터 분할하기 

In [60]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(final_feature, final_target, test_size=0.3, random_state=42)
# train, test 0.7, 0.3으로 분할한다. 

In [61]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4183, 22), (1794, 22), (4183,), (1794,))

# 손실함수 만들기 

In [62]:
from sklearn.metrics import mean_squared_error, mean_absolute_error # sklearn사용

def evaluate_regr(y,pred):
    # MAE
    mae = mean_absolute_error(y,pred)
    
    # RMSE
    rmse = np.sqrt(mean_squared_error(y,pred))
    
    # RMSLE
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    msle = np.mean((log_y - log_pred) ** 2)
    rmsle = np.sqrt(msle)    
    
    print(f'MAE: {mae:.3f}, RMSE: {rmse:.3f}, RMSLE: {rmsle:.3f}')

# Linear Regression 

In [63]:
X_train.dropna()
y_train.dropna()

3339     4.40
3878    16.46
5769     6.94
5918     8.00
1826     6.22
        ...  
3799    26.00
5231     4.57
5266     6.69
5431     8.25
865      0.70
Name: Price, Length: 4183, dtype: float64

In [64]:
from sklearn.linear_model import LinearRegression , Ridge , Lasso

lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

evaluate_regr(y_test, pred)


MAE: 5.109, RMSE: 8.897, RMSLE: 0.690


In [65]:
# lasso 규제 시도 

lasso = Lasso(alpha = 0.001)
lasso.fit(X_train, y_train)
lasso_pred_train = lasso.predict(X_train)
lasso_pred_test = lasso.predict(X_test)

In [66]:
intercept = lasso.intercept_
coef = lasso.coef_

In [67]:
coef

array([-2.27093140e-01,  2.75681632e+00, -7.22249530e-01,  4.16569527e+00,
        1.48088751e-01,  5.70319990e-01, -9.63980992e-01,  1.78507278e+00,
       -1.72584836e+00, -0.00000000e+00, -9.31966786e-01,  6.76482708e-01,
        5.31321794e+00, -0.00000000e+00, -0.00000000e+00, -1.10322048e-01,
        1.27186294e+01, -3.58155984e-14,  2.17279477e+00, -1.83443674e+00,
       -0.00000000e+00,  0.00000000e+00])

In [68]:
# R squared
print('Lasso train R-squared: ',lasso.score(X_train, y_train))
print('Lasso test R-squared : ',lasso.score(X_test, y_test))
# MSE
print('Lasso train MSE      : ',mean_squared_error(y_train, lasso_pred_train))
print('Lasso test MSE       : ',mean_squared_error(y_test, lasso_pred_test))

Lasso train R-squared:  0.4315210558739795
Lasso test R-squared :  0.4306166597394845
Lasso train MSE      :  68.05049847974058
Lasso test MSE       :  79.12959303913213
